In [1]:
import os
os.chdir("../")
%pwd

'/home/ken/Work/xray-lung-classifier'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    unzip_dir: Path
    source_url: str
    local_data_file: Path


In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
# from cnnClassifier.entity.config_entity import DataIngestionConfig


class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    
    

In [10]:
import zipfile
import os
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
# from cnnClassifier.entity.config_entity import DataIngestionConfig
import gdown

class DataIngestion:
    def __init__(self, config=DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_url
            prefix = 'https://drive.google.com/uc?/export=download&id='
            zip_download_dir = self.config.local_data_file
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            logger.info(f"Download url {prefix+file_id}")
            
            gdown.download(prefix+file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            logger.error(f"Error!!! \n {e}")
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-26 22:12:55,992: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-26 22:12:55,993: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-26 22:12:55,994: INFO: common: created directory at: artifacts]
[2024-02-26 22:12:55,995: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-26 22:12:55,995: INFO: 1165482211: Downloading data from https://drive.google.com/file/d/1wzVrA8WP-bCy5CvgHpKy74ESR1JoyvN4/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-02-26 22:12:55,996: INFO: 1165482211: Download url https://drive.google.com/uc?/export=download&id=1wzVrA8WP-bCy5CvgHpKy74ESR1JoyvN4]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1wzVrA8WP-bCy5CvgHpKy74ESR1JoyvN4
To: /home/ken/Work/xray-lung-classifier/artifacts/data_ingestion/data.zip
100%|██████████| 7.90M/7.90M [00:00<00:00, 8.95MB/s]

[2024-02-26 22:13:00,197: INFO: 1165482211: Downloaded data from https://drive.google.com/file/d/1wzVrA8WP-bCy5CvgHpKy74ESR1JoyvN4/view?usp=sharing into file artifacts/data_ingestion/data.zip]
